In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from datetime import timedelta
import requests
from pandas.tseries.offsets import BDay
from scipy.signal import cwt, ricker
import pywt

### Collect data for the last 14 years - from 2010 to 2023 in daily granularity an Identify top 10 outliers in each year.

In [2]:
def calculate_daily_returns(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    df['abs_daily_return'] = df['daily_return'].abs()
    return df

def get_top_outliers(df, n=30):
    return df.nlargest(n, 'abs_daily_return')

# Define the API key and base URL
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

# Define the currency pairs and years
pair = "C:USDCHF"
years = range(2010, 2024)

# Initialize DataFrames
stats_data = pd.DataFrame(columns=['Year', 'Mean', 'Variance', 'Skewness', 'Kurtosis'])
full_data = pd.DataFrame()
outliers_data = pd.DataFrame()

# Loop over each year
for year in years:
	# Format the API endpoint
	start_date = f'{year}-01-01'
	end_date = f'{year}-12-31'
	url = f"https://api.polygon.io/v2/aggs/ticker/{pair}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
	
	# Make the API request
	response = requests.get(url)
	data = response.json()
	
	# Check if the request was successful
	if response.status_code == 200 and 'results' in data:
		# Load data into a DataFrame
		df = pd.DataFrame(data['results'])
        # Converting timestamps
		df['date'] = pd.to_datetime(df['t'], unit='ms')
		df.drop(columns=['t'], inplace=True)
        # Calculating returns
		df = calculate_daily_returns(df)
        
		# Calculate basic statistical properties of the daily returns
		returns = df['daily_return'] 
		mean_return = returns.mean()
		variance_return = returns.var()
		skewness_return = skew(returns)
		kurtosis_return = kurtosis(returns)

		# Collect stats data by year
		year_stats = pd.DataFrame([[year, mean_return, variance_return, skewness_return, kurtosis_return]],
									columns=['Year', 'Mean', 'Variance', 'Skewness', 'Kurtosis'])
		stats_data = pd.concat([stats_data, year_stats], ignore_index=True)

		# Append the data to the full_data DataFrame for the current currency pair
		df['year'] = year
		full_data = pd.concat([full_data, df], ignore_index=True)

		# Find the top 10 outliers based on absolute values of the daily return value
		df['abs_daily_return'] = df['c'].abs()
		outliers = df.nlargest(30, 'abs_daily_return')

		# Append outliers to the outliers_data DataFrame for the current currency pair
		outliers['year'] = year
		outliers_data = pd.concat([outliers_data, outliers], ignore_index=True)

# Sort the data by date so that visualization could be done in a better way         
sorted_full_data = full_data.sort_values(by="date")
sorted_outliers_data = outliers_data.sort_values(by="date")

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/1429334715.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_data = pd.concat([stats_data, year_stats], ignore_index=True)


In [3]:
sorted_full_data

,v,vw,o,c,h,l,n,date,prev_close,daily_return,abs_daily_return,year
0,2359,1.0351,1.036260,1.036040,1.036880,1.033780,2359,2010-01-03,NaN,NaN,NaN,2010
1,69913,1.0329,1.036060,1.028910,1.041960,1.026210,69913,2010-01-04,1.03604,-0.006882,0.006882,2010
2,76822,1.0304,1.029000,1.033410,1.035960,1.025150,76822,2010-01-05,1.02891,0.004374,0.004374,2010
3,81057,1.0321,1.033410,1.027460,1.037830,1.024800,81057,2010-01-06,1.03341,-0.005758,0.005758,2010
4,72488,1.0321,1.027440,1.033700,1.037050,1.024140,72488,2010-01-07,1.02746,0.006073,0.006073,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
4399,112192,0.8555,0.856000,0.853720,0.857960,0.852800,112192,2023-12-26,0.85581,-0.002442,0.002442,2023
4400,174820,0.8495,0.853590,0.842500,0.854870,0.840600,174820,2023-12-27,0.85372,-0.013142,0.013142,2023
4401,196178,0.8392,0.842400,0.843370,0.845145,0.833000,196178,2023-12-28,0.84250,0.001033,0.001033,2023
4402,214081,0.8403,0.843460,0.840710,0.843560,0.835400,214081,2023-12-29,0.84337,-0.003154,0.003154,2023


In [4]:
sorted_outliers_data

,v,vw,o,c,h,l,n,date,prev_close,daily_return,abs_daily_return,year
28,173397,1.1107,1.103700,1.11776,1.11990,1.10215,173397,2010-05-05,1.10369,0.012748,1.11776,2010
29,147843,1.1134,1.109290,1.11767,1.11921,1.10762,147843,2010-05-13,1.10935,0.007500,1.11767,2010
26,182256,1.1240,1.117720,1.13124,1.13316,1.11499,182256,2010-05-14,1.11767,0.012141,1.13124,2010
24,6046,1.1329,1.134390,1.13310,1.13471,1.13190,6046,2010-05-16,1.13124,0.001644,1.13310,2010
25,219929,1.1360,1.133100,1.13252,1.14450,1.12966,219929,2010-05-17,1.13310,-0.000512,1.13252,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
393,154890,0.9418,0.942230,0.94085,0.94384,0.93879,154890,2023-03-08,0.94223,-0.001465,0.94085,2023
405,156286,0.9374,0.940870,0.93288,0.94157,0.93194,156286,2023-03-09,0.94085,-0.008471,0.93288,2023
406,257964,0.9220,0.914419,0.93126,0.93430,0.91200,257964,2023-03-15,0.91450,0.018327,0.93126,2023
412,253047,0.9289,0.931260,0.92900,0.93221,0.92290,253047,2023-03-16,0.93126,-0.002427,0.92900,2023


### Calculating Thresholds

In [5]:
def fetch_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data:", response.status_code, response.text)
        return None
    data = response.json()
    if 'results' not in data:
        print("No 'results' key in response:", data)
        return None
    return data

def calculate_daily_returns_threshold(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    # print(df['daily_return'])
    df['abs_daily_return'] = df['daily_return'].abs()
    return df

def convert_timestamps(df):
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)
    return df


def fetch_daily_return_prior(symbol, current_date, previous_date, api_key):
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{previous_date}/{current_date}?adjusted=true&sort=asc&apiKey={api_key}'
    data = fetch_data(url)
    if data and 'results' in data and len(data['results']) > 0:
        df = pd.DataFrame(data['results'])
        # print(df)
        df = convert_timestamps(df)
        # print(f"converted: {df}")
        df = df.loc[::-1].reset_index(drop=True)
        df = calculate_daily_returns_threshold(df)
        print(f"daily return: {df}")
        # print(f"daily_return {df['abs_daily_return']}")
        if len(df['abs_daily_return']) == 2:
            return df['abs_daily_return'].iloc[1]
    return None
    

def find_prior_outlier_threshold(df, symbol, api_key):
    df = df.sort_values(by='date').reset_index(drop=True)
    df['date'] = pd.to_datetime(df['date'])
    
    results = []
    
    for index, row in df.iterrows():
        current_date = row['date']
        current_return = row['abs_daily_return']
        print(f"outlier date: {current_date}")
        while True:
            # print(f"current_return: {current_return}")
            previous_date = current_date - pd.DateOffset(days=1)
            # print(f"previous date: {previous_date}")
            current_date_str = current_date.strftime('%Y-%m-%d')
            previous_date_str = previous_date.strftime('%Y-%m-%d')
            previous_return = fetch_daily_return_prior(symbol, current_date_str, previous_date_str, api_key)
            # print(f"previous return: {previous_return}")
            if previous_return is not None:
                if (current_return - previous_return) < 0:
                    
                    days_difference = (row['date'] - previous_date).days
                    # print(current_return - previous_return)
                    results.append({
                        'outlier_date': row['date'],
                        'prior_outlier_threshold_date': previous_date,
                        'days_difference': days_difference
                    })
                    # print()
                    break
                else:
                    current_date = previous_date
                    current_return = previous_return
            else:
                # print(f"Failed to fetch data for {previous_date_str}")
                # break
                current_date = previous_date

    return pd.DataFrame(results)


# Applying the corrected function to the data
prior_outlier_thresholds = find_prior_outlier_threshold(sorted_outliers_data, pair, api_key)


outlier date: 2010-05-05 00:00:00
daily return:         v      vw        o        c       h        l       n       date  \
0  173397  1.1107  1.10370  1.11776  1.1199  1.10215  173397 2010-05-05   
1  136874  1.0947  1.08545  1.10369  1.1050  1.08400  136874 2010-05-04   

   prev_close  daily_return  abs_daily_return  
0         NaN           NaN               NaN  
1     1.11776     -0.012588          0.012588  
daily return:         v      vw        o        c        h       l       n       date  \
0  136874  1.0947  1.08545  1.10369  1.10500  1.0840  136874 2010-05-04   
1   76325  1.0839  1.07681  1.08545  1.08881  1.0768   76325 2010-05-03   

   prev_close  daily_return  abs_daily_return  
0         NaN           NaN               NaN  
1     1.10369     -0.016526          0.016526  
outlier date: 2010-05-13 00:00:00
daily return:         v      vw        o        c        h        l       n       date  \
0  147843  1.1134  1.10929  1.11767  1.11921  1.10762  147843 2010-05-13  

In [6]:
def fetch_daily_return_post(symbol, current_date, post_date, api_key):
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{current_date}/{post_date}?adjusted=true&sort=asc&apiKey={api_key}'
    data = fetch_data(url)
    if data and 'results' in data and len(data['results']) > 0:
        df = pd.DataFrame(data['results'])
        # print(df)
        df = convert_timestamps(df)
        # print(f"converted: {df}")
        df = calculate_daily_returns_threshold(df)
        # print(f"daily return: {df}")
        # print(f"daily_return {df['abs_daily_return']}")
        if len(df['abs_daily_return']) == 2:
            return df['abs_daily_return'].iloc[1]
    return None


def find_post_outlier_threshold(df, symbol, api_key):
    df = df.sort_values(by='date').reset_index(drop=True)
    df['date'] = pd.to_datetime(df['date'])
    
    results = []
    
    for index, row in df.iterrows():
        current_date = row['date']
        current_return = row['abs_daily_return']
        # print(f"outlier date: {current_date}")
        while True:
            # print(f"current_return: {current_return}")
            post_date = current_date + pd.DateOffset(days=1)
            # print(f"previous date: {previous_date}")
            current_date_str = current_date.strftime('%Y-%m-%d')
            post_date_str = post_date.strftime('%Y-%m-%d')
            post_return = fetch_daily_return_post(symbol, current_date_str, post_date_str, api_key)
            # print(f"post return: {post_return}")
            if post_return is not None:
                if (current_return - post_return) < 0:
                    
                    days_difference = (post_date - row['date']).days
                    # print(current_return - post_return)
                    results.append({
                        'outlier_date': row['date'],
                        'post_outlier_threshold_date': post_date,
                        'days_difference': days_difference
                    })
                    # print()
                    break
                else:
                    current_date = post_date
                    current_return = post_return
            else:
                # print(f"Failed to fetch data for {post_date_str}")
                # break
                current_date = post_date

    return pd.DataFrame(results)

post_outlier_thresholds = find_post_outlier_threshold(sorted_outliers_data, pair, api_key)

No 'results' key in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '2ee3b44c5675e7526ba59f82db04aa52'}
No 'results' key in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'fba846cdb275d41d15b757c930d9087c'}


In [7]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
prior_outlier_thresholds.rename(columns={'outlier_date': 'date'}, inplace=True)
prior_outlier_thresholds

,date,prior_outlier_threshold_date,days_difference
0,2010-05-05,2010-05-03,2
1,2010-05-13,2010-05-09,4
2,2010-05-14,2010-05-09,5
3,2010-05-16,2010-05-09,7
4,2010-05-17,2010-05-13,4
...,...,...,...
415,2023-03-08,2023-03-06,2
416,2023-03-09,2023-03-06,3
417,2023-03-15,2023-03-12,3
418,2023-03-16,2023-03-14,2


In [8]:
post_outlier_thresholds

,outlier_date,post_outlier_threshold_date,days_difference
0,2010-05-05,2010-05-07,2
1,2010-05-13,2010-05-18,5
2,2010-05-14,2010-05-18,4
3,2010-05-16,2010-05-18,2
4,2010-05-17,2010-05-20,3
...,...,...,...
415,2023-03-08,2023-03-10,2
416,2023-03-09,2023-03-12,3
417,2023-03-15,2023-03-17,2
418,2023-03-16,2023-03-19,3


In [9]:
prior_outlier_thresholds.rename(columns={'outlier_date': 'date'}, inplace=True)
sorted_outliers_data = pd.merge(
    sorted_outliers_data,
    prior_outlier_thresholds[['date', 'days_difference']],
    on = 'date',
    how='left'
)
sorted_outliers_data.rename(columns={'days_difference':'pre_threshold'}, inplace=True)

post_outlier_thresholds.rename(columns={'outlier_date': 'date'}, inplace=True)
sorted_outliers_data = pd.merge(
    sorted_outliers_data,
    post_outlier_thresholds[['date', 'days_difference']],
    on = 'date',
)
sorted_outliers_data.rename(columns={'days_difference':'post_threshold'}, inplace=True)

In [10]:
sorted_outliers_data

,v,vw,o,c,h,l,n,date,prev_close,daily_return,abs_daily_return,year,pre_threshold,post_threshold
0,173397,1.1107,1.103700,1.11776,1.11990,1.10215,173397,2010-05-05,1.10369,0.012748,1.11776,2010,2,2
1,147843,1.1134,1.109290,1.11767,1.11921,1.10762,147843,2010-05-13,1.10935,0.007500,1.11767,2010,4,5
2,182256,1.1240,1.117720,1.13124,1.13316,1.11499,182256,2010-05-14,1.11767,0.012141,1.13124,2010,5,4
3,6046,1.1329,1.134390,1.13310,1.13471,1.13190,6046,2010-05-16,1.13124,0.001644,1.13310,2010,7,2
4,219929,1.1360,1.133100,1.13252,1.14450,1.12966,219929,2010-05-17,1.13310,-0.000512,1.13252,2010,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,154890,0.9418,0.942230,0.94085,0.94384,0.93879,154890,2023-03-08,0.94223,-0.001465,0.94085,2023,2,2
416,156286,0.9374,0.940870,0.93288,0.94157,0.93194,156286,2023-03-09,0.94085,-0.008471,0.93288,2023,3,3
417,257964,0.9220,0.914419,0.93126,0.93430,0.91200,257964,2023-03-15,0.91450,0.018327,0.93126,2023,3,2
418,253047,0.9289,0.931260,0.92900,0.93221,0.92290,253047,2023-03-16,0.93126,-0.002427,0.92900,2023,2,3


### Collect data of prior and post for each of these outliers.

In [11]:
# Convert dates in dataset to datetime objects
sorted_outliers_data['date'] = pd.to_datetime(sorted_outliers_data['date'])

# date_ranges = pd.DataFrame({
#     "start_date": sorted_outliers_data['date'] - BDay(3),
#     "end_date": sorted_outliers_data['date'] + BDay(3),
#     "outlier_date": sorted_outliers_data['date'],
#     "year": sorted_outliers_data['year']
# })

date_ranges = pd.DataFrame({
    "start_date": sorted_outliers_data.apply(lambda row: row['date'] - BDay(row['pre_threshold']), axis=1),
    "end_date": sorted_outliers_data.apply(lambda row: row['date'] + BDay(row['post_threshold']), axis=1),
    "outlier_date": sorted_outliers_data['date'],
    "year": sorted_outliers_data['year']
})

date_ranges

,start_date,end_date,outlier_date,year
0,2010-05-03,2010-05-07,2010-05-05,2010
1,2010-05-07,2010-05-20,2010-05-13,2010
2,2010-05-07,2010-05-20,2010-05-14,2010
3,2010-05-06,2010-05-18,2010-05-16,2010
4,2010-05-11,2010-05-20,2010-05-17,2010
...,...,...,...,...
415,2023-03-06,2023-03-10,2023-03-08,2023
416,2023-03-06,2023-03-14,2023-03-09,2023
417,2023-03-10,2023-03-17,2023-03-15,2023
418,2023-03-14,2023-03-21,2023-03-16,2023


### Calculate Daily returns and Recovery classification

In [12]:
def classify_recovery_range(df, start_date, end_date, fast_threshold=4):
    # Filter the data for the specified range
    range_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    if range_data.empty:
        return "no data"
    
    # Calculate cumulative percentage change
    cumulative_change = range_data['pct_change'].sum()
    
    # Classify the recovery based on the cumulative change
    if cumulative_change >= fast_threshold:
        return 'fast recovery'
    else:
        return 'slow recovery'

def calculate_daily_returns(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    df['abs_daily_return'] = df['daily_return'].abs()
    df['pct_change'] = df['c'].pct_change() * 100
    
    df['direction'] = df['pct_change'].apply(lambda x: 'Up' if x > 0 else 'Down')
    df['pct_change'] = df['pct_change'].abs()
    
    return df


### Fetch Hourly Data

In [13]:
def fetch_hourly_data_chunk(symbol, start_date, end_date, api_key):
    formatted_start_date = start_date.strftime('%Y-%m-%d')
    formatted_end_date = end_date.strftime('%Y-%m-%d')

    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/hour/{formatted_start_date}/{formatted_end_date}?apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return None
    
    response_data = response.json()
    
    if 'results' not in response_data:
        print(f"No 'results' in response: {response_data}")
        return None

    df = pd.DataFrame(response_data['results'])
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)
    
    return df

def fetch_and_process_hourly_data(symbol, start_date, end_date, api_key):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    
    # Split the date range into smaller chunks
    chunk_size = 3  # Fetch data in 7-day chunks
    date_ranges = [(start_date + timedelta(days=i*chunk_size), 
                    min(end_date, start_date + timedelta(days=(i+1)*chunk_size - 1)))
                   for i in range((end_date - start_date).days // chunk_size + 1)]

    # print((end_date - start_date).days // chunk_size + 1)
    all_data = []

    for start, end in date_ranges:
        chunk_data = fetch_hourly_data_chunk(symbol, start, end, api_key)
        if chunk_data is not None:
            all_data.append(chunk_data)
    
    if not all_data:
        print("No data fetched")
        return None
    
    df = pd.concat(all_data)
    hourly_data = calculate_daily_returns(df)
    hourly_data['Recovery'] = classify_recovery_range(hourly_data, start_date, end_date)
    hourly_data.set_index('date', inplace=True)
    
    full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
    hourly_data = hourly_data.reindex(full_index)
    
    hourly_data.reset_index(inplace=True)
    hourly_data.rename(columns={'index': 'date'}, inplace=True)
    
    return hourly_data

### Final Database Preparation

In [14]:
# Initialize empty DataFrame
all_data_daily = pd.DataFrame()

# Initialize an outlier identifier starting from 1 or any specific number
outlier_id = 1

for index, row in date_ranges.iterrows():
    # Convert start_date, end_date, and outlier_date to Timestamp for consistent comparison
    start_date_ts = pd.Timestamp(row['start_date'])
    end_date_ts = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)  # Extend the end date by one additional day
    outlier_date_ts = pd.Timestamp(row['outlier_date'])
    
    # Get hourly data for the range including 3 days before and after the outlier
    daily_data = fetch_and_process_hourly_data(pair, start_date_ts, end_date_ts, api_key)
    # print(daily_data)
    # Check if hourly_data is not None before processing
    if daily_data is not None:
        # Assign the current outlier_id to the data
        daily_data['outlier_id'] = outlier_id

        # Filter out weekdends
        daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5,6])]
        
        # prior_data from start_date to outlier_date inclusive
        prior_data = daily_data[(daily_data['date'] >= start_date_ts) & (daily_data['date'] < outlier_date_ts)]
        prior_data["day type"] = "prior day"

        # outlier_data is for the hourly data on the day of the outlier
        outlier_data = daily_data[(daily_data['date'].dt.date == outlier_date_ts.date())]
        outlier_data["day type"] = "outlier day"
        
        # post_data from the day after outlier_date to end_date
        post_outlier_ts = outlier_date_ts + pd.Timedelta(days=1)  # Starting the day after the outlier_date
        post_data = daily_data[(daily_data['date'] >= post_outlier_ts) & (daily_data['date'] <= end_date_ts)]
        post_data["day type"] = "post day"
        
        # Concatenate the data from this iteration to the cumulative DataFrame
        all_data_daily = pd.concat([all_data_daily, prior_data, outlier_data, post_data])

        # Increment the outlier_id for the next iteration
        outlier_id += 1

    else:
        print(f"Data not available for symbol {pair} from {row['start_date'].date()} to {row['end_date'].date()}")

# Add the day column to the final DataFrame
all_data_daily['day'] = all_data_daily['date'].dt.day_name()
# Optionally, you can reset the index of the final DataFrame if it becomes non-unique after concatenations
all_data_daily.reset_index(drop=True, inplace=True)

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/3049785879.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

No 'results' in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '406eeee917602dcba530eeac7312b2d3'}


/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/3049785879.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

No 'results' in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '629ab1e9822f1e7bfba6ecd5e145101d'}


/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/3049785879.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

...

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/3049785879.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

No 'results' in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '6f9dcbf9ff1a319cfdc37d64c37edc4f'}


/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/3049785879.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

No 'results' in response: {'ticker': 'C:USDCHF', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'b4814247b586e1e086245ae238af0a7b'}


/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/3049785879.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prior_data["day type"] = "prior day"
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/4059811168.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [15]:
# Reset the index to have simple numbers and separate "date" and "time" columns
all_data_daily.reset_index(inplace=True)
all_data_daily.rename(columns={'index': 'New'}, inplace=True)

# Convert date to string format if necessary
all_data_daily['date'] = all_data_daily['date'].astype(str)

# Extract date and time from the date string
all_data_daily['Date'] = all_data_daily['date'].str[:10]
all_data_daily['Time'] = all_data_daily['date'].str[11:19]

# Drop the 'date' column if it's no longer needed
all_data_daily.drop(columns=['date', 'New', 'v'], inplace=True)

In [16]:
len(all_data_daily)

79395

In [17]:
# # Select the column to transform
# signal = all_data_daily['pct_change'].dropna()

# all_data_daily.fillna(method='ffill', inplace=True)
# all_data_daily.fillna(method='bfill', inplace=True)

# # Define the wavelet function and scales
# wavelet = 'gaus2'
# scales = np.arange(1, 25)  # Using 24 scales as discussed

# # Compute the Continuous Wavelet Transform
# coefficients, frequencies = pywt.cwt(signal, scales, wavelet)

# # For simplicity, we'll use the mean of the coefficients across all scales as a feature
# cwt_feature = np.mean(coefficients, axis=0)

# # Since the CWT reduces the length of the data due to edge effects, align it:
# aligned_cwt_feature = np.full(all_data_daily.shape[0], np.nan)  # Initialize with NaN
# aligned_cwt_feature[:len(cwt_feature)] = cwt_feature  # Fill with CWT results

# # Add to data
# all_data_daily['CWT_Mean'] = aligned_cwt_feature

In [18]:
missing_values = all_data_daily.isnull()
missing_summary = all_data_daily.isnull().sum()

# Display summary of missing values
print("\nSummary of Missing Values:")
print(missing_summary)


Summary of Missing Values:
vw                  2054
o                   2054
c                   2054
h                   2054
l                   2054
n                   2054
prev_close          2474
daily_return        2474
abs_daily_return    2474
pct_change          2474
direction           2054
Recovery            2054
outlier_id             0
day type               0
day                    0
Date                   0
Time                   0
dtype: int64


In [19]:
all_data_daily.fillna(method='ffill', inplace=True)
all_data_daily.fillna(method='bfill', inplace=True)

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/1621982748.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data_daily.fillna(method='ffill', inplace=True)
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_96610/1621982748.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data_daily.fillna(method='bfill', inplace=True)


In [20]:
all_data_daily.head()

,vw,o,c,h,l,n,prev_close,daily_return,abs_daily_return,pct_change,direction,Recovery,outlier_id,day type,day,Date,Time
0,1.0793,1.07681,1.08041,1.08088,1.07680,1818.0,1.08041,0.002462,0.002462,0.246203,Down,fast recovery,1,prior day,Monday,2010-05-03,00:00:00
1,1.0821,1.08042,1.08307,1.08399,1.07964,3457.0,1.08041,0.002462,0.002462,0.246203,Up,fast recovery,1,prior day,Monday,2010-05-03,01:00:00
2,1.0833,1.08306,1.08235,1.08434,1.08232,1541.0,1.08307,-0.000665,0.000665,0.066478,Down,fast recovery,1,prior day,Monday,2010-05-03,02:00:00
3,1.0833,1.08236,1.08385,1.08421,1.08231,1101.0,1.08235,0.001386,0.001386,0.138587,Up,fast recovery,1,prior day,Monday,2010-05-03,03:00:00
4,1.0837,1.08385,1.08330,1.08468,1.08295,2104.0,1.08385,-0.000507,0.000507,0.050745,Down,fast recovery,1,prior day,Monday,2010-05-03,04:00:00


In [21]:
all_data_daily.to_csv("USDCHF_hourly_30.csv")